In [1]:
from transformers import BertModel, BertTokenizer, DPRQuestionEncoder, DPRQuestionEncoderTokenizer, RobertaModel, RobertaTokenizer
from sentence_transformers import SentenceTransformer
from langchain.embeddings.base import Embeddings
import torch

class BertEmbeddings:
    def __init__(self, model_name='bert-base-uncased'):
        self.tokenizer = BertTokenizer.from_pretrained(model_name)
        self.model = BertModel.from_pretrained(model_name)

    def embeddings(self, text: str):
        inputs = self.tokenizer(text, return_tensors='pt')
        outputs = self.model(**inputs)
        return outputs.pooler_output.detach().numpy()[0].tolist()
    

class RobertaEmbeddings:
    def __init__(self, model_name='roberta-base'):
        self.tokenizer = RobertaTokenizer.from_pretrained(model_name)
        self.model = RobertaModel.from_pretrained(model_name)

    def embeddings(self, text: str):
        inputs = self.tokenizer(text, return_tensors='pt')
        outputs = self.model(**inputs)
        return outputs.pooler_output.detach().numpy()[0].tolist()
    
class SentenceBertEmbeddings:
    def __init__(self, model_name='paraphrase-MiniLM-L6-v2'):
        self.model = SentenceTransformer(model_name)

    def embeddings(self, text: str):
        return self.model.encode(text).tolist()
    
class DprEmbeddings:
    def __init__(self, model_name='facebook/dpr-question_encoder-single-nq-base'):
        self.tokenizer = DPRQuestionEncoderTokenizer.from_pretrained(model_name)
        self.model = DPRQuestionEncoder.from_pretrained(model_name)

    def embeddings(self, text: str):
        inputs = self.tokenizer(text, return_tensors='pt')
        outputs = self.model(**inputs)
        return outputs.pooler_output.detach().numpy()[0].tolist()


get_embeddings = SentenceBertEmbeddings() 

/home/cgrodrigues/.pyenv/versions/3.11.9/envs/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/cgrodrigues/.pyenv/versions/3.11.9/envs/.venv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [2]:
class DPRQuestionEncoderEmbeddings(Embeddings):
    show_progress: bool = False
    """Whether to show a tqdm progress bar. Must have `tqdm` installed."""
    
    def __init__(self, model_name: str = 'facebook/dpr-question_encoder-single-nq-base'):
        self.tokenizer = DPRQuestionEncoderTokenizer.from_pretrained(model_name)
        self.model = DPRQuestionEncoder.from_pretrained(model_name)
        
    def embed(self, texts):
        # Ensure texts is a list
        if isinstance(texts, str):
            texts = [texts]
        
        embeddings = []
        # for text in texts:
        #     inputs = self.tokenizer(text, return_tensors='pt')
        #     outputs = self.model(**inputs)
        #     embedding = outputs.pooler_output.detach().numpy()[0]
        #     embeddings.append(embedding.tolist())
        
        # return embeddings

        if self.show_progress:
            try:
                from tqdm import tqdm

                iter_ = tqdm(texts, desc="Embeddings")
            except ImportError:
                logger.warning(
                    "Unable to show progress bar because tqdm could not be imported. "
                    "Please install with `pip install tqdm`."
                )
                iter_ = texts
        else:
            iter_ = texts

        for text in iter_:
            inputs = self.tokenizer(text, return_tensors='pt')
            outputs = self.model(**inputs)
            embedding = outputs.pooler_output.detach().numpy()[0]
            embeddings.append(embedding.tolist())
        
        return embeddings
    
    def embed_documents(self, documents):
        return self.embed(documents)
    
    def embed_query(self, query):
        return self.embed([query])[0]

In [3]:
from rag_2 import get_questions_answers_contexts
from ragas import evaluate
from ragas.metrics import (
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall
)
from langchain_community.chat_models import ChatOllama

# The list of metrics that we want to evaluate
metrics = [
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall
]

# We will use our local ollama with the LLaMA 3 model
langchain_llm =  ChatOllama(model="llama3")
langchain_embeddings = DPRQuestionEncoderEmbeddings('facebook/dpr-question_encoder-single-nq-base')

langchain_embeddings.show_progress = True

# Return the ragas dataset
rag_dataset = get_questions_answers_contexts(get_embeddings, store_name="documents-2")
print(rag_dataset)

# Return the metrics
results = evaluate(rag_dataset, metrics=metrics, llm=langchain_llm, embeddings=langchain_embeddings)
print(results)

Some weights of the model checkpoint at facebook/dpr-question_encoder-single-nq-base were not used when initializing DPRQuestionEncoder: ['question_encoder.bert_model.pooler.dense.bias', 'question_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRQuestionEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRQuestionEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Dataset({
    features: ['question', 'answer', 'contexts', 'ground_truth'],
    num_rows: 89
})


Evaluating:  54%|█████▍    | 192/356 [1:59:16<1:18:20, 28.66s/it]